In [1]:
# Import required packages
from datetime import datetime
import numpy as np
import pandas as pd
import torch

from utils.model import *
from utils.robot import Robot
from utils.settings import config as cfg
from utils.utils import *

In [2]:
panda = Robot(verbose=False)
# data generation
J_tr, P_tr, P_ts, F = load_all_data(panda)
# Build Generative model, NSF
config = {
        'subnet_width': 1400,
        'subnet_num_layers': 3,
        'num_transforms': 9,
        'lr': 2.1e-4,
        'lr_weight_decay': 2.7e-2,
        'decay_step_size': 4e4,
        'gamma': 5e-2,
        'batch_size': 128,
        'num_epochs': 10,
    }
# Neural spline flow (NSF) with 3 sample features and 5 context features
flow, optimizer, scheduler = get_flow_model(
        enable_load_model=cfg.use_pretrained,
        num_transforms=config["num_transforms"],
        subnet_width=config["subnet_width"],
        subnet_num_layers=config["subnet_num_layers"],
        lr=config["lr"],
        lr_weight_decay=config["lr_weight_decay"],
        decay_step_size=config["decay_step_size"],
        gamma=config["gamma"],
        device='cuda')
knn = get_knn(P_tr=P_tr)

F load successfully from /home/luca/ikpflow/data/panda/train/F-2500000-7-7-1.npy
Model load successfully from /home/luca/ikpflow/weights/panda/nsf.pth
knn load successfully from /home/luca/ikpflow/data/panda/train/knn-2500000-7-7-1-normFalse.pickle


In [4]:
mu = torch.zeros(size=(cfg.n,))
NUM_DATA = 100
NUM_SAMPLES = 1000

In [5]:
test(robot=panda, P_ts=P_ts[:NUM_DATA], F=F, solver=flow, knn=knn, K=NUM_SAMPLES, print_report=True)

       position errors (m)  orientation_errors (rad)
count        100000.000000             100000.000000
mean              0.026235                  1.519135
std               0.042640                  0.864285
min               0.000143                  0.084533
25%               0.007563                  0.733106
50%               0.013285                  1.519492
75%               0.027014                  2.307794
max               0.964838                  3.062079
average inference time (of 100 P): 0.12 sec.


,position errors (m),orientation_errors (rad)
0,0.016075,0.665450
1,0.006808,0.665450
2,0.022177,0.665450
3,0.001656,0.665450
4,0.003833,0.665450
...,...,...
99995,0.075019,0.736967
99996,0.155709,0.736967
99997,0.024577,0.736967
99998,0.057846,0.736967


In [4]:
nflow = get_nflow_model(flow=flow)
inference(robot=panda, P_inf=P_ts[:NUM_DATA], F=F, solver=nflow, knn=knn, K=NUM_SAMPLES, print_report=True)

       position errors (m)
count        100000.000000
mean              0.031947
std               0.113357
min               0.000019
25%               0.002447
50%               0.004964
75%               0.015272
max               1.615940
       inference time (sec) of 1000 samples
count                            100.000000
mean                               0.116700
std                                0.033995
min                                0.110000
25%                                0.110000
50%                                0.110000
75%                                0.120000
max                                0.450000


In [ ]:
iflow = get_iflow_model(flow, init_sample=mu, shrink_ratio=0.31)
test(robot=panda, P_ts=P_ts[:NUM_DATA], F=F, solver=iflow, knn=knn, K=NUM_SAMPLES, print_report=True)

In [ ]:
from ikflow.utils import set_seed
from ikflow.model_loading import get_ik_solver

In [ ]:
file_names = ['ang_errs_avg', 'ang_errs_min', 'F_avg', 'F_min', 'ikflow_ang', 'ikflow_l2', 'l2_errs_avg', 'l2_errs_min', 'n_evals']
exp_5_fig_dir = config.traj_dir + f'figs/exp_5_{datetime.now().strftime("%m%d%H%M")}/'
if not os.path.exists(path=exp_5_fig_dir):
    os.makedirs(exp_5_fig_dir)
set_seed()
num_trails = 100
num_generation = 100
num_ikflow_trails = num_generation
num_solutions = 500
# Build IKFlowSolver and set weights
ik_solver, hyper_parameters = get_ik_solver("panda__full__lp191_5.25m")
robot = ik_solver.robot
panda = Robot(verbose=False)

In [ ]:
def get_target_poses(P_ts):
    quaternions = np.zeros((len(P_ts), 4))
    quaternions[:, 0] = np.random.randn() * 2e-2 + 1
    target_poses = np.column_stack((P_ts, quaternions))
    return target_poses

In [ ]:
target_poses = get_target_poses(P_ts[:NUM_DATA])
target_poses.shape

In [ ]:
# -> unrefined solutions
l2_errs = np.zeros((len(target_poses), NUM_SAMPLES))
time_diffs = np.zeros((len(target_poses), NUM_SAMPLES))

for i, target_pose in enumerate(target_poses):
    iksols, l2_err, _, _, _, dt = ik_solver.solve(
        target_pose, 
        n=NUM_SAMPLES,
        refine_solutions=False, 
        return_detailed=True)

    iksols = iksols.detach().cpu().numpy()
    dt = np.zeros_like(l2_err) + dt
    
    l2_errs[i] = l2_err
    time_diffs[i] = dt

l2_errs = l2_errs.reshape((-1))
time_diffs = time_diffs.reshape((-1))

    
df = pd.DataFrame(
    data=np.column_stack((l2_errs, time_diffs)),
    columns=["l2_err", "time_diff"],
)

In [ ]:
df.describe()